In [5]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 1 : Imports et Configuration
# ════════════════════════════════════════════════════════════════════

import psycopg2
from psycopg2.extras import execute_values
import pandas as pd
import os
from dotenv import load_dotenv
from pathlib import Path

load_dotenv('../config/.env')

# Configuration RDS
RDS_CONFIG = {
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT', 5432),
    'database': os.getenv('DB_NAME', 'kayak'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD')
}

print("✅ Configuration chargée")

✅ Configuration chargée


In [4]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 2 : Import des Villes
# ════════════════════════════════════════════════════════════════════

print("📍 IMPORT DES VILLES\n")
print("="*60)

# Charger les données
cities_df = pd.read_csv('../data/processed/top5_destinations.csv')

# Connexion
conn = psycopg2.connect(**RDS_CONFIG)
cursor = conn.cursor()

# Préparer les données
cities_data = []
for _, row in cities_df.iterrows():
    cities_data.append((
        row['city'],
        row.get('latitude'),
        row.get('longitude'),
        row['avg_weather_score'],
        row['temp_avg'],
        row['rain'],
        row['humidity'],
        row['wind_speed'],
        row['clouds']
    ))

# Insert
insert_query = """
INSERT INTO cities (city_name, latitude, longitude, avg_weather_score, 
                   temp_avg, rain, humidity, wind_speed, clouds)
VALUES %s
ON CONFLICT (city_name) DO UPDATE SET
    latitude = EXCLUDED.latitude,
    longitude = EXCLUDED.longitude,
    avg_weather_score = EXCLUDED.avg_weather_score,
    temp_avg = EXCLUDED.temp_avg,
    rain = EXCLUDED.rain,
    humidity = EXCLUDED.humidity,
    wind_speed = EXCLUDED.wind_speed,
    clouds = EXCLUDED.clouds;
"""

execute_values(cursor, insert_query, cities_data)
conn.commit()

print(f"✅ {len(cities_data)} villes importées")

# Afficher les villes
cursor.execute("SELECT city_id, city_name, avg_weather_score FROM cities ORDER BY avg_weather_score DESC;")
for row in cursor.fetchall():
    print(f"   • {row[1]:25s} (ID: {row[0]}, Score: {row[2]:.2f})")

📍 IMPORT DES VILLES

✅ 5 villes importées
   • Aix en Provence           (ID: 1, Score: 76.33)
   • Marseille                 (ID: 2, Score: 76.17)
   • Bormes les Mimosas        (ID: 3, Score: 76.00)
   • Cassis                    (ID: 4, Score: 73.67)
   • Avignon                   (ID: 5, Score: 71.83)


In [6]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 2 : Chargement des Données CSV
# ════════════════════════════════════════════════════════════════════

print("📂 CHARGEMENT DES DONNÉES CSV\n")
print("="*60)

# Chemins des fichiers
csv_files = {
    'cities': '../data/processed/top5_destinations.csv',
    'hotels': '../data/processed/final_recommendations.csv',
}

# Charger les CSV
dataframes = {}

for name, path in csv_files.items():
    # Essayer différents chemins
    possible_paths = [path, f'../{path}']
    
    loaded = False
    for p in possible_paths:
        if Path(p).exists():
            df = pd.read_csv(p)
            dataframes[name] = df
            print(f"✅ {name:15s} : {len(df):,} lignes ({Path(p).stat().st_size/1024:.1f} KB)")
            loaded = True
            break
    
    if not loaded:
        print(f"❌ {name:15s} : Fichier non trouvé")

if len(dataframes) == 2:
    print(f"\n✅ Tous les fichiers chargés")
else:
    print(f"\n⚠️  {len(dataframes)}/2 fichiers chargés")

📂 CHARGEMENT DES DONNÉES CSV

✅ cities          : 5 lignes (0.6 KB)
✅ hotels          : 75 lignes (42.9 KB)

✅ Tous les fichiers chargés


In [7]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 3 : Import des Villes
# ════════════════════════════════════════════════════════════════════

print("📍 IMPORT DES VILLES\n")
print("="*60)

if 'cities' not in dataframes:
    print("❌ Données des villes non disponibles")
else:
    cities_df = dataframes['cities']
    
    print(f"📊 Aperçu des données :")
    print(cities_df.head())
    
    try:
        conn = psycopg2.connect(**RDS_CONFIG)
        cursor = conn.cursor()
        
        # Nettoyer la table
        cursor.execute("DELETE FROM cities;")
        cursor.execute("ALTER SEQUENCE cities_city_id_seq RESTART WITH 1;")
        
        # Préparer les données
        cities_data = []
        for _, row in cities_df.iterrows():
            cities_data.append((
                row['city'],
                row.get('latitude'),
                row.get('longitude'),
                row['avg_weather_score'],
                row['temp_avg'],
                row['rain'],
                row['humidity'],
                row['wind_speed'],
                row['clouds']
            ))
        
        # Insert
        insert_query = """
        INSERT INTO cities (
            city_name, latitude, longitude, avg_weather_score, 
            temp_avg, rain, humidity, wind_speed, clouds
        )
        VALUES %s
        RETURNING city_id, city_name;
        """
        
        cursor.execute("BEGIN;")
        result = execute_values(cursor, insert_query, cities_data, fetch=True)
        conn.commit()
        
        print(f"\n✅ {len(cities_data)} villes importées\n")
        
        # Afficher les villes avec leurs IDs
        cursor.execute("""
            SELECT city_id, city_name, avg_weather_score, temp_avg
            FROM cities 
            ORDER BY avg_weather_score DESC;
        """)
        
        print("📋 Villes importées :\n")
        for city_id, city_name, weather_score, temp in cursor.fetchall():
            print(f"   {city_id}. {city_name:25s} | Score: {weather_score:.2f} | Temp: {temp:.1f}°C")
        
        cursor.close()
        conn.close()
    
    except psycopg2.Error as e:
        print(f"❌ Erreur lors de l'import : {e}")
        if conn:
            conn.rollback()

📍 IMPORT DES VILLES

📊 Aperçu des données :
                 city  avg_weather_score  temp_avg  pop  rain   humidity  \
0     Aix en Provence              76.33      13.0  0.0  0.00  70.423611   
1           Marseille              76.17      15.1  3.3  0.15  67.215278   
2  Bormes les Mimosas              76.00      13.0  0.0  0.00  75.313889   
3              Cassis              73.67      14.3  7.2  0.34  67.840278   
4             Avignon              71.83      12.9  0.0  0.00  72.815278   

   wind_speed     clouds  city_id   latitude  longitude  
0    1.533194  59.155556       22  43.529842   5.447474  
1    2.989028  60.756944       21  43.296174   5.369953  
2    2.271736  50.025000       19  43.150697   6.341928  
3    2.722472  57.023611       20  43.214036   5.539632  
4    2.942667  52.477778       23  43.949249   4.805901  

✅ 5 villes importées

📋 Villes importées :

   1. Aix en Provence           | Score: 76.33 | Temp: 13.0°C
   2. Marseille                 | Score: 76.

In [8]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 5 : Import des Hôtels
# ════════════════════════════════════════════════════════════════════

print("\n\n🏨 IMPORT DES HÔTELS\n")
print("="*60)

if 'hotels' not in dataframes:
    print("❌ Données des hôtels non disponibles")
else:
    hotels_df = dataframes['hotels']
    
    print(f"📊 Nombre total d'hôtels : {len(hotels_df)}")
    print(f"\n📋 Aperçu des colonnes :")
    print(hotels_df.columns.tolist())
    
    try:
        conn = psycopg2.connect(**RDS_CONFIG)
        cursor = conn.cursor()
        
        # Récupérer le mapping city_name -> city_id
        cursor.execute("SELECT city_id, city_name FROM cities;")
        city_mapping = {row[1]: row[0] for row in cursor.fetchall()}
        
        print(f"\n🗺️  Mapping des villes :")
        for city_name, city_id in city_mapping.items():
            print(f"   • {city_name:25s} → ID {city_id}")
        
        # Nettoyer la table hotels
        cursor.execute("DELETE FROM hotels;")
        cursor.execute("ALTER SEQUENCE hotels_hotel_id_seq RESTART WITH 1;")
        
        # Préparer les données
        hotels_data = []
        skipped = 0
        
        for _, row in hotels_df.iterrows():
            city_id = city_mapping.get(row['city'])
            
            if city_id:
                # Gérer les valeurs NaN
                hotels_data.append((
                    city_id,
                    row['hotel_name'],
                    row['url'] if pd.notna(row['url']) else None,
                    row['property_type'] if pd.notna(row.get('property_type')) else None,
                    float(row['score']) if pd.notna(row['score']) else None,
                    int(row['number_of_reviews']) if pd.notna(row['number_of_reviews']) else None,
                    float(row['price']) if pd.notna(row['price']) else None,
                    row.get('currency', 'EUR'),
                    float(row['latitude']) if pd.notna(row.get('latitude')) else None,
                    float(row['longitude']) if pd.notna(row.get('longitude')) else None,
                    row.get('facilities') if pd.notna(row.get('facilities')) else None,
                    row.get('image_url') if pd.notna(row.get('image_url')) else None
                ))
            else:
                skipped += 1
        
        if skipped > 0:
            print(f"\n⚠️  {skipped} hôtels ignorés (ville non trouvée)")
        
        # Insert
        insert_query = """
        INSERT INTO hotels (
            city_id, hotel_name, url, property_type, score, 
            number_of_reviews, price, currency, latitude, longitude,
            facilities, image_url
        )
        VALUES %s
        RETURNING hotel_id;
        """
        
        print(f"\n🚀 Import de {len(hotels_data)} hôtels...")
        
        cursor.execute("BEGIN;")
        execute_values(cursor, insert_query, hotels_data)
        conn.commit()
        
        print(f"✅ {len(hotels_data)} hôtels importés")
        
        # Statistiques par ville
        cursor.execute("""
            SELECT 
                c.city_name,
                COUNT(h.hotel_id) as nb_hotels,
                AVG(h.score) as avg_score,
                AVG(h.price) as avg_price,
                MIN(h.price) as min_price,
                MAX(h.price) as max_price
            FROM cities c
            LEFT JOIN hotels h ON c.city_id = h.city_id
            GROUP BY c.city_name
            ORDER BY nb_hotels DESC;
        """)
        
        print(f"\n📊 Statistiques par ville :\n")
        print(f"{'Ville':25s} | {'Nb':3s} | {'Note':5s} | {'Prix Moy':8s} | {'Min':6s} | {'Max':6s}")
        print("-" * 75)
        
        for city, nb, score, avg_p, min_p, max_p in cursor.fetchall():
            score_str = f"{score:.1f}" if score else "N/A"
            avg_str = f"{avg_p:.0f}€" if avg_p else "N/A"
            min_str = f"{min_p:.0f}€" if min_p else "N/A"
            max_str = f"{max_p:.0f}€" if max_p else "N/A"
            print(f"{city:25s} | {nb:3d} | {score_str:5s} | {avg_str:8s} | {min_str:6s} | {max_str:6s}")
        
        cursor.close()
        conn.close()
    
    except psycopg2.Error as e:
        print(f"❌ Erreur lors de l'import : {e}")
        if conn:
            conn.rollback()



🏨 IMPORT DES HÔTELS

📊 Nombre total d'hôtels : 75

📋 Aperçu des colonnes :
['city', 'hotel_name', 'url', 'property_type', 'score', 'number_of_reviews', 'price', 'currency', 'latitude', 'longitude', 'avg_weather_score', 'temp_avg', 'rain', 'clouds', 'weather_score_norm', 'hotel_score_norm', 'price_score_norm', 'final_score', 'facilities', 'image_url', 'rank']

🗺️  Mapping des villes :
   • Aix en Provence           → ID 1
   • Marseille                 → ID 2
   • Bormes les Mimosas        → ID 3
   • Cassis                    → ID 4
   • Avignon                   → ID 5

🚀 Import de 75 hôtels...
✅ 75 hôtels importés

📊 Statistiques par ville :

Ville                     | Nb  | Note  | Prix Moy | Min    | Max   
---------------------------------------------------------------------------
Marseille                 |  15 | 8.3   | 286€     | 115€   | 725€  
Avignon                   |  15 | 8.6   | 257€     | 105€   | 736€  
Bormes les Mimosas        |  15 | 8.1   | 445€     | 169€   | 

In [12]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 6 : Import des Recommandations
# ════════════════════════════════════════════════════════════════════

print("\n\n🎯 IMPORT DES RECOMMANDATIONS\n")
print("="*60)

if 'hotels' not in dataframes:
    print("❌ Données non disponibles")
else:
    hotels_df = dataframes['hotels']
    
    # Filtrer uniquement les hôtels avec un rank
    recommendations_df = hotels_df[hotels_df['rank'].notna()].copy()
    
    print(f"📊 Recommandations brutes : {len(recommendations_df)}")
    
    # ✅ CORRECTION : Dédupliquer par (city, hotel_name) en gardant le meilleur rank
    recommendations_df = recommendations_df.sort_values('rank').drop_duplicates(
        subset=['city', 'hotel_name'],
        keep='first'
    )
    
    print(f"📊 Après déduplication : {len(recommendations_df)}")
    
    try:
        conn = psycopg2.connect(**RDS_CONFIG)
        cursor = conn.cursor()
        
        # Récupérer le mapping (city_name, hotel_name) -> hotel_id
        cursor.execute("""
            SELECT h.hotel_id, c.city_name, h.hotel_name
            FROM hotels h
            JOIN cities c ON h.city_id = c.city_id;
        """)
        
        hotel_mapping = {(row[1], row[2]): row[0] for row in cursor.fetchall()}
        
        print(f"🗺️  {len(hotel_mapping)} hôtels disponibles pour le mapping")
        
        # Nettoyer la table recommendations
        cursor.execute("DELETE FROM recommendations;")
        cursor.execute("ALTER SEQUENCE recommendations_rec_id_seq RESTART WITH 1;")
        
        # Préparer les données
        recommendations_data = []
        skipped = 0
        mapped_hotel_ids = set()  # ✅ Pour détecter les doublons
        
        for _, row in recommendations_df.iterrows():
            hotel_id = hotel_mapping.get((row['city'], row['hotel_name']))
            
            if hotel_id and hotel_id not in mapped_hotel_ids:
                recommendations_data.append((
                    hotel_id,
                    int(row['rank']) if pd.notna(row['rank']) else None,
                    float(row['weather_score_norm']) if pd.notna(row.get('weather_score_norm')) else None,
                    float(row['hotel_score_norm']) if pd.notna(row.get('hotel_score_norm')) else None,
                    float(row['price_score_norm']) if pd.notna(row.get('price_score_norm')) else None,
                    float(row['final_score']) if pd.notna(row['final_score']) else None
                ))
                mapped_hotel_ids.add(hotel_id)
            elif hotel_id in mapped_hotel_ids:
                print(f"   ⚠️  Doublon ignoré : {row['city']} - {row['hotel_name']}")
                skipped += 1
            else:
                skipped += 1
        
        if skipped > 0:
            print(f"⚠️  {skipped} recommandations ignorées (hôtel non trouvé ou doublon)")
        
        # Insert
        insert_query = """
        INSERT INTO recommendations (
            hotel_id, rank, weather_score_norm, hotel_score_norm,
            price_score_norm, final_score
        )
        VALUES %s;
        """
        
        print(f"\n🚀 Import de {len(recommendations_data)} recommandations uniques...")
        
        cursor.execute("BEGIN;")
        execute_values(cursor, insert_query, recommendations_data)
        conn.commit()
        
        print(f"✅ {len(recommendations_data)} recommandations importées")
        
        # Top 10
        cursor.execute("""
            SELECT 
                r.rank,
                c.city_name,
                h.hotel_name,
                h.score,
                h.price,
                r.final_score
            FROM recommendations r
            JOIN hotels h ON r.hotel_id = h.hotel_id
            JOIN cities c ON h.city_id = c.city_id
            ORDER BY r.rank
            LIMIT 10;
        """)
        
        print(f"\n🏆 TOP 10 RECOMMANDATIONS :\n")
        print(f"{'#':2s} | {'Ville':15s} | {'Hôtel':30s} | {'Note':4s} | {'Prix':6s} | {'Score':5s}")
        print("-" * 85)
        
        for rank, city, hotel, score, price, final in cursor.fetchall():
            hotel_short = hotel[:28] + '..' if len(hotel) > 30 else hotel
            score_str = f"{score:.1f}" if score else "N/A"
            price_str = f"{price:.0f}€" if price else "N/A"
            final_str = f"{final:.2f}" if final else "N/A"
            print(f"{rank:2d} | {city:15s} | {hotel_short:30s} | {score_str:4s} | {price_str:6s} | {final_str:5s}")
        
        cursor.close()
        conn.close()
    
    except psycopg2.Error as e:
        print(f"❌ Erreur lors de l'import : {e}")
        if conn:
            conn.rollback()



🎯 IMPORT DES RECOMMANDATIONS

📊 Recommandations brutes : 75
📊 Après déduplication : 74
🗺️  74 hôtels disponibles pour le mapping

🚀 Import de 74 recommandations uniques...
✅ 74 recommandations importées

🏆 TOP 10 RECOMMANDATIONS :

#  | Ville           | Hôtel                          | Note | Prix   | Score
-------------------------------------------------------------------------------------
 1 | Aix en Provence | La Villa Rustica               | 9.8  | 234€   | 9.86 
 2 | Bormes les Mimosas | Le Mas de la Palmeraie - Stu.. | 9.7  | 198€   | 9.57 
 3 | Aix en Provence | Les Suites du Cours & Spa      | 9.2  | 398€   | 9.45 
 4 | Bormes les Mimosas | La Favière proche de la plage  | 9.5  | 242€   | 9.44 
 5 | Bormes les Mimosas | Boulevard du Soleil            | 9.5  | 242€   | 9.44 
 6 | Aix en Provence | Domaine de Saint Clair         | 9.6  | 561€   | 9.44 
 7 | Marseille       | RockyPop Marseille Hôtel       | 9.1  | 252€   | 9.42 
 8 | Marseille       | Loft 200m2 avec écran le

In [15]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 7 : Import des Données Météo Historiques
# ════════════════════════════════════════════════════════════════════

print("🌤️ IMPORT DES DONNÉES MÉTÉO HISTORIQUES\n")
print("="*60)

try:
    # Charger les données météo brutes
    weather_file = '../data/processed/city_weather_scores.csv'
    possible_paths = [weather_file, f'../{weather_file}']
    
    weather_df = None
    for path in possible_paths:
        if Path(path).exists():
            weather_df = pd.read_csv(path)
            print(f"✅ Fichier météo chargé : {len(weather_df)} villes")
            break
    
    if weather_df is None:
        print(f"❌ Fichier météo non trouvé : {weather_file}")
    else:
        # Afficher un aperçu
        print(f"\n📊 Aperçu des données :")
        print(weather_df.head())
        
        conn = psycopg2.connect(**RDS_CONFIG)
        cursor = conn.cursor()
        
        # Récupérer le mapping city_name -> city_id
        cursor.execute("SELECT city_id, city_name FROM cities;")
        city_mapping = {row[1]: row[0] for row in cursor.fetchall()}
        
        print(f"\n🗺️  {len(city_mapping)} villes disponibles pour le mapping")
        
        # Nettoyer la table
        cursor.execute("DELETE FROM weather_history;")
        cursor.execute("ALTER SEQUENCE weather_history_weather_id_seq RESTART WITH 1;")
        
        # Date d'enregistrement (date actuelle ou date du fichier)
        date_recorded = pd.Timestamp.now().date()
        
        # Préparer les données
        weather_data = []
        skipped = 0
        
        for _, row in weather_df.iterrows():
            city_id = city_mapping.get(row['city'])
            
            if city_id:
                weather_data.append((
                    city_id,
                    date_recorded,
                    float(row['temp_avg']) if pd.notna(row.get('temp_avg')) else None,
                    float(row['rain']) if pd.notna(row.get('rain')) else None,
                    None,  # sun_hours (non disponible dans vos données)
                    float(row['wind_speed']) if pd.notna(row.get('wind_speed')) else None
                ))
            else:
                skipped += 1
        
        if skipped > 0:
            print(f"⚠️  {skipped} enregistrements ignorés (ville non trouvée)")
        
        # Insert
        insert_query = """
        INSERT INTO weather_history (
            city_id, date_recorded, temp_avg, precipitation, 
            sun_hours, wind_speed
        )
        VALUES %s;
        """
        
        print(f"\n🚀 Import de {len(weather_data)} enregistrements météo...")
        
        cursor.execute("BEGIN;")
        execute_values(cursor, insert_query, weather_data)
        conn.commit()
        
        print(f"✅ {len(weather_data)} enregistrements importés")
        
        # Afficher les données importées
        cursor.execute("""
            SELECT 
                c.city_name,
                w.date_recorded,
                w.temp_avg,
                w.precipitation,
                w.wind_speed
            FROM weather_history w
            JOIN cities c ON w.city_id = c.city_id
            ORDER BY c.city_name;
        """)
        
        print(f"\n📊 Données météo importées :\n")
        print(f"{'Ville':20s} | {'Date':10s} | {'Temp':6s} | {'Pluie':6s} | {'Vent':6s}")
        print("-" * 60)
        
        for city, date, temp, rain, wind in cursor.fetchall():
            temp_str = f"{temp:.1f}°C" if temp else "N/A"
            rain_str = f"{rain:.1f}mm" if rain else "N/A"
            wind_str = f"{wind:.1f}m/s" if wind else "N/A"
            print(f"{city:20s} | {date} | {temp_str:6s} | {rain_str:6s} | {wind_str:6s}")
        
        cursor.close()
        conn.close()

except FileNotFoundError:
    print(f"❌ Fichier non trouvé : {weather_file}")
except psycopg2.Error as e:
    print(f"❌ Erreur lors de l'import : {e}")
    if conn:
        conn.rollback()
except Exception as e:
    print(f"❌ Erreur inattendue : {e}")

🌤️ IMPORT DES DONNÉES MÉTÉO HISTORIQUES

✅ Fichier météo chargé : 35 villes

📊 Aperçu des données :
                 city  avg_weather_score  temp_avg  pop  rain   humidity  \
0     Aix en Provence              76.33      13.0  0.0  0.00  70.423611   
1           Marseille              76.17      15.1  3.3  0.15  67.215278   
2  Bormes les Mimosas              76.00      13.0  0.0  0.00  75.313889   
3              Cassis              73.67      14.3  7.2  0.34  67.840278   
4             Avignon              71.83      12.9  0.0  0.00  72.815278   

   wind_speed     clouds  city_id   latitude  longitude  
0    1.533194  59.155556       22  43.529842   5.447474  
1    2.989028  60.756944       21  43.296174   5.369953  
2    2.271736  50.025000       19  43.150697   6.341928  
3    2.722472  57.023611       20  43.214036   5.539632  
4    2.942667  52.477778       23  43.949249   4.805901  

🗺️  5 villes disponibles pour le mapping
⚠️  30 enregistrements ignorés (ville non trouvée)

🚀

In [16]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 8 : Vérification et Test des Vues
# ════════════════════════════════════════════════════════════════════

print("🔍 VÉRIFICATION ET TEST DES VUES\n")
print("="*60)

try:
    conn = psycopg2.connect(**RDS_CONFIG)
    cursor = conn.cursor()
    
    # Test de la vue top_recommendations
    print("\n📊 Vue: top_recommendations (Top 5)")
    cursor.execute("SELECT * FROM top_recommendations LIMIT 5;")
    
    columns = [desc[0] for desc in cursor.description]
    print(f"\nColonnes : {', '.join(columns[:7])}")
    
    for row in cursor.fetchall():
        print(f"   #{row[0]:<2} {row[1]:15s} {row[2][:25]:25s} Score: {row[9]:.2f}")
    
    # Test de la vue city_statistics
    print("\n\n📊 Vue: city_statistics")
    cursor.execute("SELECT * FROM city_statistics ORDER BY avg_weather_score DESC;")
    
    print(f"\n{'Ville':20s} | {'Score Météo':11s} | {'Nb Hôtels':9s} | {'Prix Moy':8s}")
    print("-" * 60)
    
    for row in cursor.fetchall():
        city, weather_score, temp, rain, nb_hotels, avg_score, avg_price = row[:7]
        print(f"{city:20s} | {weather_score:11.2f} | {nb_hotels:9d} | {avg_price:8.0f}€")
    
    # Compter le contenu
    print("\n\n📈 STATISTIQUES FINALES\n")
    
    stats_queries = {
        'cities': 'SELECT COUNT(*) FROM cities',
        'hotels': 'SELECT COUNT(*) FROM hotels',
        'recommendations': 'SELECT COUNT(*) FROM recommendations',
        'weather_history': 'SELECT COUNT(*) FROM weather_history'
    }
    
    for table, query in stats_queries.items():
        cursor.execute(query)
        count = cursor.fetchone()[0]
        print(f"   • {table:20s} : {count:,} lignes")
    
    # Taille de la base
    cursor.execute("SELECT pg_size_pretty(pg_database_size(current_database()));")
    db_size = cursor.fetchone()[0]
    print(f"\n   💾 Taille totale : {db_size}")
    
    cursor.close()
    conn.close()

except psycopg2.Error as e:
    print(f"❌ Erreur : {e}")

🔍 VÉRIFICATION ET TEST DES VUES


📊 Vue: top_recommendations (Top 5)

Colonnes : rank, city_name, hotel_name, hotel_score, price, currency, number_of_reviews
   #1  Aix en Provence La Villa Rustica          Score: 9.86
   #2  Bormes les Mimosas Le Mas de la Palmeraie -  Score: 9.57
   #3  Aix en Provence Les Suites du Cours & Spa Score: 9.45
   #4  Bormes les Mimosas La Favière proche de la p Score: 9.44
   #5  Bormes les Mimosas Boulevard du Soleil       Score: 9.44


📊 Vue: city_statistics

Ville                | Score Météo | Nb Hôtels | Prix Moy
------------------------------------------------------------
Aix en Provence      |       76.33 |        15 |      293€
Marseille            |       76.17 |        15 |      286€
Bormes les Mimosas   |       76.00 |        15 |      445€
Cassis               |       73.67 |        15 |      373€
Avignon              |       71.83 |        15 |      257€


📈 STATISTIQUES FINALES

   • cities               : 5 lignes
   • hotels              

In [17]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 9 : Exemples de Requêtes Utiles
# ════════════════════════════════════════════════════════════════════

print("💡 EXEMPLES DE REQUÊTES UTILES\n")
print("="*60)

try:
    conn = psycopg2.connect(**RDS_CONFIG)
    cursor = conn.cursor()
    
    # 1. Meilleur rapport qualité/prix
    print("\n1️⃣  MEILLEUR RAPPORT QUALITÉ/PRIX (Prix < 200€)\n")
    cursor.execute("""
        SELECT 
            c.city_name,
            h.hotel_name,
            h.score,
            h.price,
            r.final_score
        FROM hotels h
        JOIN cities c ON h.city_id = c.city_id
        LEFT JOIN recommendations r ON h.hotel_id = r.hotel_id
        WHERE h.price < 200 AND h.score >= 8.0
        ORDER BY r.final_score DESC NULLS LAST, h.score DESC
        LIMIT 5;
    """)
    
    for city, hotel, score, price, final in cursor.fetchall():
        final_str = f"{final:.2f}" if final else "N/A"
        print(f"   • {hotel[:35]:35s} | {city:15s} | {score:.1f}/10 | {price:.0f}€ | Score: {final_str}")
    
    # 2. Hôtels par tranche de prix
    print("\n\n2️⃣  RÉPARTITION PAR TRANCHE DE PRIX\n")
    cursor.execute("""
        SELECT 
            CASE 
                WHEN price < 100 THEN 'Économique (< 100€)'
                WHEN price < 200 THEN 'Modéré (100-200€)'
                WHEN price < 300 THEN 'Confort (200-300€)'
                ELSE 'Luxe (> 300€)'
            END as categorie,
            COUNT(*) as nb_hotels,
            AVG(score) as avg_score
        FROM hotels
        WHERE price IS NOT NULL
        GROUP BY categorie
        ORDER BY MIN(price);
    """)
    
    for categorie, nb, avg_score in cursor.fetchall():
        print(f"   • {categorie:25s} : {nb:2d} hôtels | Note moyenne: {avg_score:.1f}/10")
    
    # 3. Top 3 par ville
    print("\n\n3️⃣  TOP 3 HÔTELS PAR VILLE\n")
    cursor.execute("""
        SELECT 
            c.city_name,
            h.hotel_name,
            h.score,
            h.price
        FROM (
            SELECT 
                city_id,
                hotel_name,
                score,
                price,
                ROW_NUMBER() OVER (PARTITION BY city_id ORDER BY score DESC) as rn
            FROM hotels
            WHERE score IS NOT NULL
        ) h
        JOIN cities c ON h.city_id = c.city_id
        WHERE h.rn <= 3
        ORDER BY c.city_name, h.score DESC;
    """)
    
    current_city = None
    for city, hotel, score, price in cursor.fetchall():
        if city != current_city:
            print(f"\n   📍 {city.upper()}")
            current_city = city
        print(f"      • {hotel[:40]:40s} | {score:.1f}/10 | {price:.0f}€")
    
    cursor.close()
    conn.close()

except psycopg2.Error as e:
    print(f"❌ Erreur : {e}")

💡 EXEMPLES DE REQUÊTES UTILES


1️⃣  MEILLEUR RAPPORT QUALITÉ/PRIX (Prix < 200€)

   • Le Mas de la Palmeraie - Studio dan | Bormes les Mimosas | 9.7/10 | 198€ | Score: 9.57
   • Studio climatisé plein centre 4eme  | Aix en Provence | 8.4/10 | 174€ | Score: 9.36
   • Aparthotel Adagio Access Marseille  | Marseille       | 8.2/10 | 149€ | Score: 9.16
   • Hotel La Voile                      | Bormes les Mimosas | 8.4/10 | 169€ | Score: 9.07
   • Entre les pins et les oliviers      | Bormes les Mimosas | 8.4/10 | 198€ | Score: 9.04


2️⃣  RÉPARTITION PAR TRANCHE DE PRIX

   • Modéré (100-200€)         : 27 hôtels | Note moyenne: 8.1/10
   • Confort (200-300€)        : 21 hôtels | Note moyenne: 8.8/10
   • Luxe (> 300€)             : 26 hôtels | Note moyenne: 8.2/10


3️⃣  TOP 3 HÔTELS PAR VILLE


   📍 AIX EN PROVENCE
      • La Villa Rustica                         | 9.8/10 | 234€
      • Domaine de Saint Clair                   | 9.6/10 | 561€
      • Villa Saint-Ange                   